## Sentiment Analysis

In [74]:
import time 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import nltk 
import io 
import unicodedata 
import re 
import string 
from numpy import linalg 
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import PunktSentenceTokenizer 
from nltk.corpus import webtext 
from nltk.stem.porter import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 
from textblob import TextBlob 
from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [75]:
#!pip install textblob

## Read the coronavirus Tweets.csv file and set it as a Dataframe called covid_tweets. Check the head, info, and describe methods on covid_tweets 

In [76]:
covid_tweets=pd.read_csv(r"C:\Users\Administrator\Downloads\Coronavirus Tweets.csv", encoding='latin-1')

In [77]:
covid_tweets.head() 

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


In [78]:
covid_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.9+ MB


In [79]:
covid_tweets.describe()

,UserName,ScreenName,Sentiment
count,41157.000000,41157.000000,41157.000000
mean,24377.000000,69329.000000,0.813276
std,11881.146851,11881.146851,0.725727
min,3799.000000,48751.000000,0.000000
25%,14088.000000,59040.000000,0.000000
50%,24377.000000,69329.000000,1.000000
75%,34666.000000,79618.000000,1.000000
max,44955.000000,89907.000000,2.000000


## Remove punctuations and stopwords from the text in the ‘OriginalTweet’ column. Replace special symbols with ‘_’(underscore). Convert all the letters to lowercase.

In [80]:
covid_tweets.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [81]:
covid_tweets['OriginalTweet']

0        @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1        advice Talk to your neighbours family to excha...
2        Coronavirus Australia: Woolworths to give elde...
3        My food stock is not the only one which is emp...
4        Me, ready to go at supermarket during the #COV...
                               ...                        
41152    Airline pilots offering to stock supermarket s...
41153    Response to complaint not provided citing COVI...
41154    You know itÂs getting tough when @KameronWild...
41155    Is it wrong that the smell of hand sanitizer i...
41156    @TartiiCat Well new/used Rift S are going for ...
Name: OriginalTweet, Length: 41157, dtype: object

In [82]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [84]:
covid_tweets.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [85]:
covid_tweets['tweet_without_stopwords'] = covid_tweets['OriginalTweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [86]:
covid_tweets['tweet_without_stopwords'][0]

'@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa https://t.co/xX6ghGFzCC https://t.co/I2NlzdxNo8'

In [87]:
punct = "\n\r"+string.punctuation
covid_tweets['tweet_without_stopwords']= covid_tweets['tweet_without_stopwords'].str.translate(str.maketrans('','',punct))


In [88]:
covid_tweets['tweet_without_stopwords'][0]

'MeNyrbie PhilGahan Chrisitv httpstcoiFz9FAn2Pa httpstcoxX6ghGFzCC httpstcoI2NlzdxNo8'

In [89]:
punct = "\n\r"+string.punctuation
punct

'\n\r!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [90]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
#convert all letters to lowercase

covid_tweets['tweet_without_stopwords']= covid_tweets['tweet_without_stopwords'].map(lambda s:s.lower() if type(s) == str else s)

In [92]:
covid_tweets.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment', 'tweet_without_stopwords'],
      dtype='object')

In [93]:
covid_tweets['tweet_without_stopwords']

0        menyrbie philgahan chrisitv httpstcoifz9fan2pa...
1        advice talk neighbours family exchange phone n...
2        coronavirus australia woolworths give elderly ...
3        my food stock one empty please panic there wil...
4        me ready go supermarket covid19 outbreak not i...
                               ...                        
41152    airline pilots offering stock supermarket shel...
41153    response complaint provided citing covid19 rel...
41154    you know itâs getting tough kameronwilds rati...
41155    is wrong smell hand sanitizer starting turn on...
41156    tartiicat well newused rift s going 70000 amaz...
Name: tweet_without_stopwords, Length: 41157, dtype: object

## Normalize tweets by using Stemming or Lemmatization.

In [94]:
# stemming

In [95]:
st = PorterStemmer()
covid_tweets['tweet_without_stopwords']= covid_tweets['tweet_without_stopwords'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [96]:
covid_tweets['tweet_without_stopwords']

0        menyrbi philgahan chrisitv httpstcoifz9fan2pa ...
1        advic talk neighbour famili exchang phone numb...
2        coronaviru australia woolworth give elderli di...
3        my food stock one empti pleas panic there will...
4        me readi go supermarket covid19 outbreak not i...
                               ...                        
41152    airlin pilot offer stock supermarket shelv nz ...
41153    respons complaint provid cite covid19 relat de...
41154    you know itâ get tough kameronwild ration toi...
41155    is wrong smell hand sanit start turn on corona...
41156    tartiicat well newus rift s go 70000 amazon rn...
Name: tweet_without_stopwords, Length: 41157, dtype: object

## Preprocessed tweets should be included in the covid_tweets data frame as cleaned_tweets.

In [97]:
covid_tweets.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment', 'tweet_without_stopwords'],
      dtype='object')

In [98]:
covid_tweets.rename(columns={'tweet_without_stopwords':'cleaned_tweets'},inplace=True)

In [99]:
covid_tweets.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,cleaned_tweets
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2,menyrbi philgahan chrisitv httpstcoifz9fan2pa ...
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1,advic talk neighbour famili exchang phone numb...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1,coronaviru australia woolworth give elderli di...
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1,my food stock one empti pleas panic there will...
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0,me readi go supermarket covid19 outbreak not i...


## Create two objects X and y. X will be the ' cleaned_tweets ' column of covid_tweets data frame and y will be the ' Sentiment' column

In [100]:
X=covid_tweets.cleaned_tweets

In [101]:
y=covid_tweets.Sentiment

In [102]:
covid_tweets.head(2)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,cleaned_tweets
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2,menyrbi philgahan chrisitv httpstcoifz9fan2pa ...
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1,advic talk neighbour famili exchang phone numb...


## create a CountVectorizer object and split the data into training and testing sets. Perform TFIDF vectorization on Text feature “cleaned_tweets’

In [103]:
# CountVectorizer is a great tool provided by the scikit-learn library in Python.
#It is used to transform a given text into a vector on the basis of the frequency (count),
#of each word that occurs in the entire text

In [110]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 7500)
X = cv.fit_transform(covid_tweets['cleaned_tweets']).toarray()
y = covid_tweets.Sentiment.values

In [111]:
len(X[0])

7500

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Train a decision tree model for classifying the headlines and Displaying the confusion Matrix.

In [113]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [114]:
y_pred = classifier.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [115]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2066  693  321]
 [ 672 2658  296]
 [ 232  250 1044]]


0.7006802721088435